In [1]:
import pandas as pd
from covid_data_helper import *


In [11]:
import requests
import re
from bs4 import BeautifulSoup

url = 'https://www.kff.org/report-section/state-data-and-policy-actions-to-address-coronavirus-sources/'
res = requests.get(url)
html_page = res.content

res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

output = ''
blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script']

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)


rawlist = list(map(str.strip, output.split('\n')))
outlist = []
this_state = ''
# us_states = [x.upper() for x in us_state_abbrev.keys()]
us_states = {k.upper():v for k,v in us_state_abbrev.items()}

for linenum in range(len(rawlist)):
    thisstr = rawlist[linenum]
    print(thisstr)
    if thisstr.strip().upper() in us_states.keys():
        change_dir = 'restricting'
        this_state_abbrev = us_states[thisstr.strip().upper()]
        this_state = abbrev_us_state[this_state_abbrev]
    elif thisstr.strip()[:6] == 'Easing':
        change_dir = 'easing'
    elif this_state != '' and thisstr.strip()[:4] not in ['http','']:
        
#         idx_before_urls = thisstr.find(':')
        idx_before_urls = re.search("[A-Za-z]", thisstr).start()
        
        dates = re.findall(r'\d+/\d+', thisstr[:idx_before_urls])
        if len(dates) > 0:
            last_dt = dates[-1]

            ld_idx = thisstr.find(last_dt)

            name_url = thisstr[ld_idx+len(last_dt):]
            l_name_url = re.split(":", name_url, 1)

            name = l_name_url[0].strip()
            if len(l_name_url) > 1:
                urls = l_name_url[1].strip()

            outlist.append([this_state, this_state_abbrev, dates[0]+'/2020', dates, name, change_dir, urls])

df_out = pd.DataFrame(outlist, 
                      columns=['state', 'state_code', 'dt', 'all_dates', 
                               'event_name', 'social_distancing_direction', 'urls'])
df_out['dt'] = pd.to_datetime(df_out['dt'])

df_out

State Data and Policy Actions to Address Coronavirus – Sources | KFF
img.wp-smiley,
img.emoji {
display: inline !important;
border: none !important;
box-shadow: none !important;
height: 1em !important;
width: 1em !important;
margin: 0 .07em !important;
vertical-align: -0.1em !important;
background: none !important;
padding: 0 !important;
}

.has-text-align-justify{text-align:justify;}

.wpcom-related-posts ul li {
list-style-type: none;
display: inline-block;
}
img#wpstats{display:none}
/* If html does not have either class, do not show lazy loaded images. */
html:not( .jetpack-lazy-images-js-enabled ):not( .js ) .jetpack-lazy-image {
display: none;
}

Google Tag Manager


End Google Tag Manager


Home
Topics

Coronavirus (COVID-19)
Disparities Policy
Global Health Policy
Health Costs
Health Reform
HIV/AIDS
Medicaid
Medicare
Private Insurance
Uninsured
Women’s Health Policy


Polling
Perspectives
State Health Facts

State Health Facts
Custom State Reports


Graphics & Interactives
Char

,state,state_code,dt,all_dates,event_name,social_distancing_direction,urls
0,Alabama,AL,2020-03-13,"[3/13, 5/8]",Public Health Emergency,restricting,https://governor.alabama.gov/newsroom/2020/03/...
1,Alabama,AL,2020-03-17,"[3/17, 3/19, 3/20, 3/26, 3/27, 4/28]","School Closures, Large Gatherings Ban, Bar/Res...",restricting,http://www.alabamapublichealth.gov/legal/asset...
2,Alabama,AL,2020-03-18,[3/18],Primary Election Postponement,restricting,https://governor.alabama.gov/newsroom/2020/03/...
3,Alabama,AL,2020-03-23,[3/23],Section 1135 Waiver,restricting,https://www.medicaid.gov/state-resource-center...
4,Alabama,AL,2020-03-27,"[3/27, 4/3]","Non-Essential Business Closures, Stay At Home ...",restricting,http://www.alabamapublichealth.gov/legal/asset...
...,...,...,...,...,...,...,...
549,Wyoming,WY,2020-03-27,[3/27],Section 1135 Waiver,restricting,https://www.medicaid.gov/state-resource-center...
550,Wyoming,WY,2020-04-03,"[4/3, 4/29]",Mandatory Quarantine for Travelers,restricting,
551,Wyoming,WY,2020-05-15,"[5/15, 6/1]",Bar/Restaurant Limits (announced 5/13),easing,
552,Wyoming,WY,2020-05-15,"[5/15, 6/1, 6/15]",Large Gatherings Ban (announced 5/13),easing,


In [16]:
df_sum = df_out.groupby(['state_code','social_distancing_direction']).count().head(105)


In [10]:
df_out.to_csv('./output/kff_policy_events_{}.csv'.format(pd.Timestamp.today().strftime("%Y%m%d")),
    encoding='utf-8')